<a href="https://colab.research.google.com/github/rgranit/academix-ydata-project/blob/master/code/2_metric_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before we can calculate Precision/Recall, we need a way to match ID's with clustering group

Psuedo-Code:

Sort PI-ID's by most PMID's assosciated with name.

For pi in PI-IDs:<br>
> Find cluster, `c`, which has the largest number of pi.<br>
If `c.label` is None, c.label = pi.<br>
Otherwise, iterate through clusters until find cluster without label.<br>

Precision = number of pi where c.label = pi / # of articles in given c
Recall = number of pi where c.label = pi / # of pi not in c

If for a pi there is no c:
Precision/Recall = 0

Mis-integration:
For every cluster, count the number of dif PI IDs assigned to cluster.

Mis-Separation:
For every PI ID, count the number of dif clusters it is assigned to.


## Installation

In [0]:
!pip install boto

     |████████████████████████████████| 1.4MB 4.9MB/s 


In [0]:
import numpy as np
import pandas as pd
from collections import Counter 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from collections import defaultdict
import s3_function

In [0]:
df= s3_functions.get_dataframe_from_s3(file="enriched_labeled_dataset.csv")

# Implementation

As we want to test the theory of the metrics without actually running any clustering algorith, I will make some toy data.

In [0]:
df = df[["pmid","PI_IDS"]]

In [0]:
top_5_ids = df.groupby('PI_IDS').size().sort_values(ascending=False).reset_index()['PI_IDS'][:5]
df_top = df[df['PI_IDS'].isin(list(top_5_ids))]

In [0]:
np.random.seed(42)
df_top['cluster_pred'] = np.array(np.random.randint(0,5,size=(len(df_top))))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


We have 274 items based off 5 authors, are for arguments sake we made K=5, and randomly assigned a cluster to each item to simulate an ineffiecient clustering algo.

In [0]:
# Creating a dictionary where each cluster is assigned an ID based off psuedo-code written above K_dict[pi_id] = # of cluster
K_dict = dict()
set_of_clusters = set(df_top.cluster_pred)
#get pi_ids sorted by most productive
pi_id_sorted = list(df_top.groupby('PI_IDS').size().sort_values(ascending=False).reset_index()['PI_IDS'])
for pi_id in pi_id_sorted:
  #Get most popular clusters for each id
  cluster_for_id = [c_id for c_id, _ in Counter(df_top[df_top['PI_IDS'] == pi_id].cluster_pred).most_common()]
  for c in cluster_for_id:
    if c not in set_of_clusters:
      continue
    else:
      K_dict[pi_id] = c
      set_of_clusters.remove(c)
      break

df_top['cluster_max'] = [K_dict[pid] for pid in df_top.PI_IDS]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


With each cluster being assigned a ID, we can now 'check' our guess and see metrics such as Precision/Recall and mis-integration and mis-separation.

In [0]:
print("Precision score: {}, Recall score: {}".format(precision_score(df_top.cluster_max,df_top.cluster_pred,average='micro'),recall_score(df_top.cluster_max,df_top.cluster_pred,average='micro')))

Precision score: 0.22262773722627738, Recall score: 0.22262773722627738


In [0]:
mis_intergration_dict = defaultdict(int)
mis_separation_dict = defaultdict(int)

In [0]:
#mis_separation

for clus_sep in df_top.groupby('PI_IDS')['cluster_pred'].nunique():
  mis_separation_dict[clus_sep] += 1

mis_separation_dict = dict(mis_separation_dict)

total = sum(mis_separation_dict.values())
for key in mis_separation_dict.keys():
  mis_separation_dict[key] /= total

print(mis_separation_dict)

In [0]:
#mis-integration - for each cluster, see how many dif people were attributed to it

for clus_int in df_top.groupby('cluster_max')['cluster_pred'].nunique():
  mis_intergration_dict[clus_int] += 1

mis_intergration_dict = dict(mis_intergration_dict)

total = sum(mis_intergration_dict.values())
for key in mis_intergration_dict.keys():
  mis_intergration_dict[key] /= total

print(mis_intergration_dict)

{5: 1.0}


**Still needs to be discussed when # of clusters is larger and smaller than the #  of pi_ids**